# 华为云询价API调用

## 库导入

In [ ]:
# coding: utf-8

from typing import Any
from huaweicloudsdkcore.auth.credentials import GlobalCredentials
from huaweicloudsdkbssintl.v2.region.bssintl_region import BssintlRegion
from huaweicloudsdkcore.exceptions import exceptions
from huaweicloudsdkbssintl.v2 import *
import os
from openpyxl import Workbook
from openpyxl import load_workbook
import xlrd

def print_to_file(content,filename): #输出到文件，防止终端输出截断
    with open(filename,'w',encoding='utf-8') as f:
        print(content,file=f)

## 敏感信息

In [ ]:
ak = "your ak"
sk = "your sk"

## 主体部分(Class版本)

调用时不能使用华为内网，否则鉴权错误

In [ ]:
class query_price():

    def __init__(self, ak, sk, language, region) -> None:
        self.__ak=ak
        self.__sk=sk
        self.language=language
        self.region=region

        region_code_dict={'singapore':'ap-southeast-3','hong kong':'ap-southeast-1'}
        project_id_dict={'singapore':'05680d57900025632f1ec0133b6411ac'}

        self.language='en_US' if('english'==self.language.lower()) else 'zh_CN'
        self.region=region_code_dict[self.region.lower()]
        self.project_id=project_id_dict[region.lower()]
        self.measure_unit={}

        self.__credentials = GlobalCredentials(self.__ak, self.__sk)
        self.__client = BssintlClient.new_builder() \
        .with_credentials(self.__credentials) \
        .with_region(BssintlRegion.value_of("ap-southeast-1")) \
        .build()

        measure_id_request = ListMeasureUnitsRequest()
        measure_id_request.x_language=self.language #默认为中文，可设置语言为英文en_US/中文zh_CN
        self.measure_id_response = self.__client.list_measure_units(measure_id_request)

        self.OnDemandProductList()
        self.OnPeriodProductList()

    def __call__(self) :
        #按需查询
        self.RequestSending('OnDemand')
        #包月包年查询
        self.RequestSending('OnPeriod')

    def RequestSending(self,RequestType):
        match RequestType:
            case 'OnDemand':
                try:
                    request = ListOnDemandResourceRatingsRequest()
                    request.body = RateOnDemandReq(
                        product_infos=self.OnDemandListProductInfosbody,
                        inquiry_precision=1,
                        project_id=self.project_id
                    )
                    response = self.__client.list_on_demand_resource_ratings(request)
                    results=response._product_rating_results


                    print('OnDemand ==============================================================')
                    print('offical website amount:\n')


                    dic=self.excel_dict('OnDemand',results)
                    self.print_to_excel(dic)


                    for i in range(len(results)):
                        print(f'{i} {self.OnDemandListProductInfosbody[i]._id} | {self.OnDemandListProductInfosbody[i]._resource_spec}\
                              \n{results[i]._official_website_amount} {response._currency} per {self.OnDemandListProductInfosbody[i]._usage_value} {self.get_usage_measure_name_by_measure_id(self.OnDemandListProductInfosbody[i]._usage_measure_id)}\n=================================================')
                except exceptions.ClientRequestException as e:
                    print(e.status_code)
                    print(e.request_id)
                    print(e.error_code)
                    print(e.error_msg)

            case 'OnPeriod':
                try:
                    request = ListRateOnPeriodDetailRequest()
                    request.body = RateOnPeriodReq(
                        product_infos=self.OnPeriodListProductInfosbody,
                        project_id=self.project_id
                    )
                    response = self.__client.list_rate_on_period_detail(request)
                    results=response._official_website_rating_result._product_rating_results


                    print('OnPeriod ==============================================================')
                    print('offical website amount:\n')

                    dic=self.excel_dict('OnPeriod',results)
                    self.print_to_excel(dic)

                    period_type={2:'month',3:'year'}
                    for i in range(len(results)):
                        print(f'{i} {self.OnPeriodListProductInfosbody[i]._id} | {self.OnPeriodListProductInfosbody[i]._resource_spec}\
                              \n{results[i]._official_website_amount} {response._currency} per {self.OnPeriodListProductInfosbody[i].period_num} {period_type[self.OnPeriodListProductInfosbody[i].period_type]}\n=================================================')
                except exceptions.ClientRequestException as e:
                    print(e.status_code)
                    print(e.request_id)
                    print(e.error_code)
                    print(e.error_msg)

        return

    def get_usage_measure_name_by_measure_id(self,measure_id):
        for i in self.measure_id_response._measure_units:
            if i._measure_id == measure_id:
                ans=i._measure_name
                break
        return ans

    def OnDemandProductList(self):
        self.OnDemandListProductInfosbody = [
            #ECS ==========================================
            DemandProductInfo(
                id="t6 1vcpu 1G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="t6.small.1.linux", #t6 1vcpu 1G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="t6 2vcpu 4G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="t6.large.2.linux", #t6 2vcpu 4G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="s6 2vcpu 8G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="s6.large.4.linux", #s6 2vcpu 8G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="s6 4vcpu 16G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="s6.xlarge.4.linux", #s6 4vcpu 16G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="c6 4vcpu 8G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6.xlarge.2.linux", #c6 4vcpu 8G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="c6 4vcpu 16G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6.xlarge.4.linux", #c6 4vcpu 16G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="c6s 4vcpu 16G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6s.xlarge.4.linux", #c6s 4vcpu 16G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="c6 16vcpu 64G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6.4xlarge.4.linux", #c6 16vcpu 64G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="m6 4vcpu 32G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="m6.xlarge.8.linux", #m6 4vcpu 32G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="m6 16vcpu 128G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="m6.4xlarge.8.linux", #m6 16vcpu 128G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="g5 32vcpu 128G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="g5.8xlarge.4.linux", #g5 32vcpu 128G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="pi2 8vcpu 32G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="pi2.2xlarge.4.linux", #pi2 8vcpu 32G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="p2v 8vcpu 64G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="p2v.2xlarge.8.linux", #p2v 8vcpu 64G
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            #ECS ===============================================

            #OBS ================================================
            DemandProductInfo(
                id="obs 1az 100TB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", # 
                region=self.region,
                usage_factor="size",
                usage_value=100,
                usage_measure_id=9,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs 3az 100TB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", # 
                region=self.region,
                usage_factor="size_3az",
                usage_value=100,
                usage_measure_id=47,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs 3az 1PB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", 
                region=self.region,
                usage_factor="size_3az",
                usage_value=1,
                usage_measure_id=48,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs infrequent 1az 100TB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", 
                region=self.region,
                usage_factor="size_warm",
                usage_value=100,
                usage_measure_id=9,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs infrequent 3az 100TB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", 
                region=self.region,
                usage_factor="size_warm_3az",
                usage_value=100,
                usage_measure_id=47,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs archive 1az 100TB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", 
                region=self.region,
                usage_factor="size_cold",
                usage_value=100,
                usage_measure_id=9,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs archive 1az 1PB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", 
                region=self.region,
                usage_factor="size_cold",
                usage_value=1,
                usage_measure_id=8,
                subscription_num=1
            ),
            DemandProductInfo(
                id="obs download traffic 1TB",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.001", 
                region=self.region,
                usage_factor="download.external",
                usage_value=1,
                usage_measure_id=9,
                subscription_num=1
            ),
            #OBS =================================================

            #EIP =================================================
            DemandProductInfo(
                id="eip by Bandwith 5mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_bgp",
                region=self.region,
                resource_size=5,
                size_measure_id=15,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="eip by Bandwith 20mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_bgp",
                region=self.region,
                resource_size=20,
                size_measure_id=15,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="eip by Bandwith 200mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_bgp",
                region=self.region,
                resource_size=200,
                size_measure_id=15,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            #共享带宽理论上不收费，使用by bandwith的费用，所以为同一价格



            #EIP ==================================================

            #EVS ==================================================
            DemandProductInfo(
                id="EVS 高IO 200GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SAS",
                region=self.region,
                resource_size=200,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="EVS 高IO 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SAS",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="EVS 通用型SSD 200GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="GPSSD",
                region=self.region,
                resource_size=200,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="EVS 通用型SSD 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="GPSSD",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="EVS 超高IO 100GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SSD",
                region=self.region,
                resource_size=100,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="EVS 超高IO 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SSD",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="EVS 普通IO 100GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SATA",
                region=self.region,
                resource_size=100,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            DemandProductInfo(
                id="EVS 普通IO 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SATA",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            # EVS ================================================

            # RDS ================================================
            # 规格码：https://www.huaweicloud.com/product/mysql/instance-types.html
            DemandProductInfo(
                id="rds Mysql 单机 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.mysql.n1.xlarge.4",
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
            
            DemandProductInfo(
                id="rds Mysql 主备 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.mysql.x1.xlarge.4.ha",
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="rds Mysql 主备 16cpu 64G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.mysql.x1.4xlarge.4.ha",
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="rds pg 单机 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.pg.n1.xlarge.4",
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="rds pg 主备 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.pg.n1.xlarge.4.ha",
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="rds pg 主备 16cpu 64G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.pg.x1.4xlarge.4.ha",
                region=self.region,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            # RDS =================================================
            
            # 备份 backup valut======================================
            DemandProductInfo(
                id="backup valut 200g",
                cloud_service_type="hws.service.type.cbr",
                resource_type="hws.resource.type.cbr.vault",
                resource_spec="vault.backup.server.normal",
                region=self.region,
                resource_size=200,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="backup valut 2000g",
                cloud_service_type="hws.service.type.cbr",
                resource_type="hws.resource.type.cbr.vault",
                resource_spec="vault.backup.server.normal",
                region=self.region,
                resource_size=2000,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),

            DemandProductInfo(
                id="backup valut 20000g",
                cloud_service_type="hws.service.type.cbr",
                resource_type="hws.resource.type.cbr.vault",
                resource_spec="vault.backup.server.normal",
                region=self.region,
                resource_size=20000,
                size_measure_id=17,
                usage_factor="Duration",
                usage_value=1,
                usage_measure_id=4,
                subscription_num=1
            ),
        ]
        return

    def OnPeriodProductList(self):
        self.OnPeriodListProductInfosbody=[]
        self.OnPeriodListProductInfosbody.extend([
            #ECS ==========================================
            PeriodProductInfo(
                id="t6 1vcpu 1G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="t6.small.1.linux", #t6 1vcpu 1G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="t6 2vcpu 4G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="t6.large.2.linux", #t6 2vcpu 4G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="s6 2vcpu 8G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="s6.large.4.linux", #s6 2vcpu 8G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="s6 4vcpu 16G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="s6.xlarge.4.linux", #s6 4vcpu 16G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="c6 4vcpu 8G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6.xlarge.2.linux", #c6 4vcpu 8G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="c6 4vcpu 16G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6.xlarge.4.linux", #c6 4vcpu 16G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="c6s 4vcpu 16G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6s.xlarge.4.linux", #c6s 4vcpu 16G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="c6 16vcpu 64G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="c6.4xlarge.4.linux", #c6 16vcpu 64G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="m6 4vcpu 32G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="m6.xlarge.8.linux", #m6 4vcpu 32G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="m6 16vcpu 128G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="m6.4xlarge.8.linux", #m6 16vcpu 128G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="g5 32vcpu 128G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="g5.8xlarge.4.linux", #g5 32vcpu 128G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="pi2 8vcpu 32G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="pi2.2xlarge.4.linux", #pi2 8vcpu 32G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="p2v 8vcpu 64G",
                cloud_service_type="hws.service.type.ec2",
                resource_type="hws.resource.type.vm",
                resource_spec="p2v.2xlarge.8.linux", #p2v 8vcpu 64G
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            #ECS ===============================================

            #OBS ================================================
            PeriodProductInfo(
                id="obs 1az 100TB month",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.storage.100tb",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="obs 3az 100TB month",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.storage_3AZ.100tb",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="obs download traffic 1TB 1month",
                cloud_service_type="hws.service.type.obs",
                resource_type="hws.resource.type.obs",
                resource_spec="obs.down.traffic.1tb",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            #OBS =================================================

            #EIP =================================================
            PeriodProductInfo(
                id="eip by Bandwith 5mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_bgp",
                region=self.region,
                resource_size=5,
                size_measure_id=15,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="eip by Bandwith 20mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_bgp",
                region=self.region,
                resource_size=20,
                size_measure_id=15,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="eip by Bandwith 200mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_bgp",
                region=self.region,
                resource_size=200,
                size_measure_id=15,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="eip by share Bandwith 5mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_share",
                region=self.region,
                resource_size=5,
                size_measure_id=15,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="eip by share Bandwith 20mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_share",
                region=self.region,
                resource_size=20,
                size_measure_id=15,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="eip by share Bandwith 200mbps",
                cloud_service_type="hws.service.type.vpc",
                resource_type="hws.resource.type.bandwidth",
                resource_spec="19_share",
                region=self.region,
                resource_size=200,
                size_measure_id=15,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            #共享带宽理论上不收费，使用by bandwith的费用，所以为同一价格

            #EIP ==================================================

            #EVS ==================================================
            PeriodProductInfo(
                id="EVS 高IO 200GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SAS",
                region=self.region,
                resource_size=200,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 高IO 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SAS",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 通用型SSD 200GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="GPSSD",
                region=self.region,
                resource_size=200,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 通用型SSD 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="GPSSD",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 超高IO 100GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SSD",
                region=self.region,
                resource_size=100,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 超高IO 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SSD",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 普通IO 100GB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SATA",
                region=self.region,
                resource_size=100,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            PeriodProductInfo(
                id="EVS 普通IO 1TB",
                cloud_service_type="hws.service.type.ebs",
                resource_type="hws.resource.type.volume",
                resource_spec="SATA",
                region=self.region,
                resource_size=1,
                size_measure_id=47,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            # EVS ================================================

            # RDS ================================================
            # 规格码：https://www.huaweicloud.com/product/mysql/instance-types.html
            PeriodProductInfo(
                id="rds Mysql 单机 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.mysql.n1.xlarge.4",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            
            PeriodProductInfo(
                id="rds Mysql 主备 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.mysql.x1.xlarge.4.ha",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="rds Mysql 主备 16cpu 64G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.mysql.x1.4xlarge.4.ha",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="rds pg 单机 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.pg.n1.xlarge.4",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="rds pg 主备 4cpu 16G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.pg.n1.xlarge.4.ha",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="rds pg 主备 16cpu 64G",
                cloud_service_type="hws.service.type.rds",
                resource_type="hws.resource.type.rds.vm",
                resource_spec="rds.pg.x1.4xlarge.4.ha",
                region=self.region,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
            # RDS =================================================
            
            # 备份 backup valut======================================
            PeriodProductInfo(
                id="backup valut 200g",
                cloud_service_type="hws.service.type.cbr",
                resource_type="hws.resource.type.cbr.vault",
                resource_spec="vault.backup.server.normal",
                region=self.region,
                resource_size=200,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="backup valut 2000g",
                cloud_service_type="hws.service.type.cbr",
                resource_type="hws.resource.type.cbr.vault",
                resource_spec="vault.backup.server.normal",
                region=self.region,
                resource_size=2000,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),

            PeriodProductInfo(
                id="backup valut 20000g",
                cloud_service_type="hws.service.type.cbr",
                resource_type="hws.resource.type.cbr.vault",
                resource_spec="vault.backup.server.normal",
                region=self.region,
                resource_size=20000,
                size_measure_id=17,
                period_type=2,
                period_num=1,
                subscription_num=1
            ),
        ])
        tmp_list=[]
        for info in self.OnPeriodListProductInfosbody:
            tmp=PeriodProductInfo()
            tmp.id=info.id+' by year'
            tmp.cloud_service_type=info.cloud_service_type
            tmp.resource_type=info.resource_type
            tmp.resource_spec=info.resource_spec
            tmp.region=self.region
            tmp.resource_size=info.resource_size
            tmp.size_measure_id=info.size_measure_id
            tmp.period_type=3
            tmp.period_num=info.period_num
            tmp.subscription_num=info.subscription_num
            tmp_list.append(tmp)
        self.OnPeriodListProductInfosbody.extend(tmp_list)
        return

    def excel_dict(self,QueryType,results):
        dic={}
        match QueryType:
            case 'OnDemand':
                #ECS 0-12
                for i in range(24,36):
                    dic[f'C{i}']=results[i-24]
                dic['C37']=results[12]
                #OBS 13-20
                for i in range(73,80):
                    dic[f'C{i}']=results[i-(73-13)]
                dic['C81']=results[20]
                #EIP 21-23
                for i in range(50,53):
                    dic[f'C{i}']=results[i-(50-21)]
                for i in range(53,56):
                    dic[f'C{i}']=results[i-(53-21)]
                #EVS 24-31
                for i in range(40,45):
                    dic[f'C{i}']=results[i-(40-24)]
                #RDS 32-37
                for i in range(61,67):
                    dic[f'C{i}']=results[i-(61-32)]
                #Backup 38-40
                for i in range(85,87):
                    dic[f'C{i}']=results[i-(85-38)]
                    
            case 'OnPeriod':
                #ECS 0-12
                for i in range(24,36):
                    dic[f'D{i}']=results[i-24]
                dic['D37']=results[12]
                #OBS 13-15
                dic['D73']=results[13]
                dic['D74']=results[14]
                dic['D81']=results[15]
                #EIP 16-21
                for i in range(50,56):
                    dic[f'D{i}']=results[i-(50-16)]
                #EVS 22-26
                for i in range(40,45):
                    dic[f'D{i}']=results[i-(40-22)]
                #RDS 30-35
                for i in range(61,67):
                    dic[f'D{i}']=results[i-(61-30)]
                #Backup 36-38
                for i in range(85,87):
                    dic[f'D{i}']=results[i-(85-36)]
                

                # by year
                for i in range(24,36):
                    dic[f'F{i}']=results[i-24+39]
                dic['F37']=results[12+39]
                #OBS 13-15
                dic['F73']=results[13+39]
                dic['F74']=results[14+39]
                dic['F81']=results[15+39]
                #EIP 16-21
                for i in range(50,56):
                    dic[f'F{i}']=results[i-(50-16)+39]
                #EVS 22-26
                for i in range(40,45):
                    dic[f'F{i}']=results[i-(40-22)+39]
                #RDS 30-35
                for i in range(61,67):
                    dic[f'F{i}']=results[i-(61-30)+39]
                #Backup 36-38
                for i in range(85,87):
                    dic[f'F{i}']=results[i-(85-36)+39]
                


        return dic

    def print_to_excel(self,dic):
        self.addr='test.xlsx'
        self.wb=load_workbook(self.addr)
        self.ws=self.wb['Sheet1']
        for key,value in dic.items():
            self.ws[key]=value._official_website_amount
        self.wb.save('test.xlsx')
        return


test_query=query_price(ak,sk,'English','Singapore')
test_query()

## 价格请求(示例)

In [ ]:
credentials = GlobalCredentials(ak, sk)

client = BssintlClient.new_builder() \
        .with_credentials(credentials) \
        .with_region(BssintlRegion.value_of("ap-southeast-1")) \
        .build()

try:
    request = ListOnDemandResourceRatingsRequest()
    listProductInfosbody = [
        DemandProductInfo(
            id="1",
            cloud_service_type="hws.service.type.ec2",
            resource_type="hws.resource.type.vm",
            resource_spec="t6.small.1.linux",
            region="ap-southeast-3",
            usage_factor="Duration",
            usage_value=1,
            usage_measure_id=4,
            subscription_num=1
        ),
        DemandProductInfo(
            id="2",
            cloud_service_type="hws.service.type.ec2",
            resource_type="hws.resource.type.vm",
            resource_spec="t6.small.1.win",
            region="ap-southeast-3",
            usage_factor="Duration",
            usage_value=1,
            usage_measure_id=4,
            subscription_num=1
        ),
        DemandProductInfo(
            id="3",
            cloud_service_type="hws.service.type.obs",
            resource_type="hws.resource.type.obs",
            resource_spec="obs.001",
            region="ap-southeast-3",
            usage_factor="size",
            usage_value=100,
            usage_measure_id=10,
            subscription_num=1
        ),
        DemandProductInfo(
            id="4",
            cloud_service_type="hws.service.type.obs",
            resource_type="hws.resource.type.obs",
            resource_spec="obs.001",
            region="ap-southeast-3",
            usage_factor="size",
            usage_value=100,
            usage_measure_id=10,
            subscription_num=1
        )
    ]

except exceptions.ClientRequestException as e:
    print(e.status_code)
    print(e.request_id)
    print(e.error_code)
    print(e.error_msg)

## 单位查询 (usage measure id)

In [ ]:
try:
    request = ListMeasureUnitsRequest()
    request.x_language="zh_cn" #默认为英文，可设置语言为中文
    response = client.list_measure_units(request)
    print(response)
    ans=''
    n=1
    for i in response._measure_units:
        ans+=f'{n}  {i._measure_id}   {i._measure_name}  {i._abbreviation} {i._measure_type}\n'
        n+=1
    print_to_file(ans,'usage measure id.txt')
except exceptions.ClientRequestException as e:
    print(e.status_code)
    print(e.request_id)
    print(e.error_code)
    print(e.error_msg)

## 服务类型查询 （service type）

In [ ]:
print('====================================================')
print('List Service Types')
try:
    request = ListServiceTypesRequest(limit=1000)
    response = client.list_service_types(request)
    print(response)
    ans=''
    n=1
    for i in response._service_types:
        ans+=f'{n}  {i._service_type_name}   {i._service_type_code}  {i._abbreviation}\n'
        n+=1
    print_to_file(ans,'service type.txt')
except exceptions.ClientRequestException as e:
    print(e.status_code)
    print(e.request_id)
    print(e.error_code)
    print(e.error_msg)

## 资源类型查询 （resource type）

In [ ]:
print('====================================================')
print('List Resource Types')
try:
    request = ListResourceTypesRequest(offset=0,limit=1100)
    response = client.list_resource_types(request)
    print(response)
    ans=''
    n=1
    for i in response._resource_types:
        ans+=f'{n}  {i._resource_type_code} {i._resource_type_name} {i._resource_type_desc} {i._service_type_code} \n'
        n+=1
    print_to_file(ans,'resource type list.txt')
except exceptions.ClientRequestException as e:
    print(e.status_code)
    print(e.request_id)
    print(e.error_code)
    print(e.error_msg)

# 参考文档 及参数查询方法
RDS实例规格码：https://www.huaweicloud.com/product/mysql/instance-types.html   
价格计算器：https://www.huaweicloud.com/intl/zh-cn/pricing/calculator.html#/obs  
右键检查-网络-resource 载荷 将json展开
